# Data Understanding

### Import Data

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

client = 'TOOL_CLIENT.csv'
sales  = 'TOOL_SALES.csv'
joint = 'TOOL_JOINT.csv'

In [ ]:
df_client = pd.read_csv(client)
df_sales = pd.read_csv(sales)

df = pd.merge(df_client, df_sales, on='CLIENT_ID')

# df = pd.read_csv(joint)

# lower case and replace spaces with underscores in column names
original_columns = df.columns
renamed_columns = [col.lower().replace(" ", "_") for col in original_columns]
column_mapping = dict(zip(original_columns, renamed_columns))
df.rename(columns=column_mapping, inplace=True)

df.head()

In [ ]:
df.info()

In [ ]:
# converting date columns to datetime
df['client_create_date'] = pd.to_datetime(df['client_create_date'])
df['yyyymm'] = pd.to_datetime(df['yyyymm'].astype(str), format='%Y%m')

# converting other columns to appropriate data types
df['client_id'] = df['client_id'].astype(str)
df['cancelled'] = df['cancelled'] == 'X'
df['unit'] = df['unit'] == 'P'

In [ ]:
df.info()

In [ ]:
# exporting the merged data

# df.to_csv('TOOL_JOINT.csv', index=False)

Create a new variable to identify client

Create a new variable to identify unique sales

In [ ]:
# creating a new column for identifie the unique sales
df['sales_id'] = df['client_id'].astype(str) + '_' + df['yyyymm'].dt.strftime('%Y%m')

df.head()

In [ ]:
# creating new dataframes with no canceled orders
no_canceled = df[df['cancelled'] == False]
no_canceled.head()

In [ ]:
# checking the number of unique sales per client
client_sales_count = no_canceled.groupby('client_id')['sales_id'].nunique().sort_values(ascending=False)
print(client_sales_count)

In [ ]:
client_sales_count.describe()

In [ ]:
# filtering the clients with more than one sale
client_sales_count.drop(client_sales_count[client_sales_count < 2].index, inplace=True)

pluri_client = no_canceled[no_canceled['client_id'].isin(client_sales_count.index)]
pluri_client['n_purchases'] = pluri_client.groupby('client_id')['sales_id'].transform('nunique')

print(pluri_client.shape)
print(pluri_client.info())

In [ ]:
pluri_client.head()

In [ ]:
for c in pluri_client.columns:
    print(c)
    print(pluri_client[c].value_counts())

We have now a new dataset cointaining only client with more than one purchase

In [ ]:
plt.figure(figsize=(20, 6))
sns.countplot(x='n_purchases', data=pluri_client, hue='unit')
plt.xlabel('Number of Purchases')
plt.ylabel('Number of Clients')
plt.title('Number of Purchases per Client')
plt.show()

On this point I'd like asking business about articles that aren't "Unit". I suppose that could be substitute parts for other articles or similar, which a client buys because it absolutly needs it and a marketing campaign about is unuseful. So I decide to drop rows containing non-piece units

In [ ]:
# dropping unnecessary columns
pluri_client.drop(pluri_client[pluri_client['unit'] == False].index, inplace=True)
pluri_client.drop('unit', axis=1, inplace=True)
pluri_client.drop('cancelled', axis=1, inplace=True)

pluri_client.head()

In [ ]:
pluri_client.shape

### Next steps: 
- Days since last purchase for same client
- Analyze time between purchases

- Create a DF with total amount for each sales_id

In [ ]:
# extracting new variables
sales_net = pluri_client.groupby('sales_id')['net'].sum()   # total sales per sale
sales_n_purchases = pluri_client.groupby('sales_id')['n_purchases'].max()   # number of purchases per sale

sales_time = pluri_client[['sales_id', 'client_id', 'yyyymm']].drop_duplicates().set_index('client_id')   # time of the sale
sales_time = sales_time.sort_values(by=['client_id', 'yyyymm'])

sales_time['time_diff'] = sales_time.groupby('client_id')['yyyymm'].diff().dt.days  # time between sales
sales_id_time_diff = sales_time[['sales_id', 'time_diff']].set_index('sales_id')    # time between sales per sale (to easly merge with the main dataframe)

In [ ]:
# adding the new variables to the main dataframe
sales_time = pluri_client[['sales_id', 'client_id', 'yyyymm']].drop_duplicates().set_index('client_id')   # time of the sale
sales_time = sales_time.sort_values(by=['client_id', 'yyyymm'])

sales_time['time_diff'] = sales_time.groupby('client_id')['yyyymm'].diff().dt.days  # time between sales
sales_id_time_diff = sales_time[['sales_id', 'time_diff']].set_index('sales_id')    # time between sales per sale (to easly merge with the main dataframe)

pluri_client['sales_net'] = pluri_client['sales_id'].map(sales_net)
pluri_client['time_diff'] = pluri_client['sales_id'].map(sales_id_time_diff['time_diff'].to_dict())
pluri_client.head()

In [ ]:
sales_time.info()

In [ ]:
# creating a new dataframe for sales
sales_net = sales_net.reset_index()
sales_n_purchases = sales_n_purchases.reset_index()
sales_time.reset_index(inplace=True)

merged = pd.merge(sales_time, sales_net, on='sales_id')
sales = pd.merge(merged, sales_n_purchases, on='sales_id')

In [ ]:
# adding the new variables to the sales dataframe
sales_columns = ['region', 'trade_sector', 'n_employees', 'economic_pot',
                 'eco_pot_class', 'risk_cat', 'flg_tool', 'sales_channel', 'sales_id']  # columns to be added to the sales dataframe

for c in sales_columns: # adding the columns to the sales dataframe
    col = pluri_client.groupby('sales_id')[c].first()
    sales = pd.merge(sales, col, left_on='sales_id', right_index=True)

sales.drop(['sales_id_x', 'sales_id_y'], axis=1, inplace=True)

In [ ]:
sales.head(10)

P.S. de-comment the follow cell to save files

In [ ]:
sales.to_csv('sales.csv', index=False)
pluri_client.to_csv('TOOLS_PREP.csv', index=False)

Now we have 2 DataFrame to use for our analysis
- df: the complete DataFrame with all the variables
- sales: the DataFrame with the details of sales

# Association rules

## Working on association rules

In [ ]:
# one-hot encoding the itemsets for the apriori algorithm
itemsets_series = pluri_client.groupby('client_id')['item_id'].apply(list)
encoder = TransactionEncoder()
itemsets = itemsets_series.tolist()

onehot = encoder.fit_transform(itemsets, sparse=True)
onehot = pd.DataFrame.sparse.from_spmatrix(onehot, columns = encoder.columns_)
onehot.columns = [str(col) for col in onehot.columns]
onehot.info()

frequent_itemsets = apriori(onehot, min_support=0.01, max_len = 4, use_colnames=True, low_memory=True)
rules = association_rules(frequent_itemsets, metric='confidence', min_threshold=0.2)
print(len(rules))

In [ ]:
# filtering and ordering the rules
rules.sort_values(by=['support', 'confidence'], ascending=False, inplace=True)
print('We have a total of {} rules'.format(rules.shape[0]))
rules.head(20)